In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [2]:
# Normalization values for CIFAR-10
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2
)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2
)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels,
                               kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(out_channels, out_channels,
                               kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut (projection) if shape changes (stride != 1 or channels differ)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out


In [4]:
class ResNet56(nn.Module):
    def __init__(self, block=BasicBlock, num_classes=10):
        super(ResNet56, self).__init__()
        self.in_channels = 16

        # Initial conv: 3x3, 16 filters
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        # Each layer: 9 blocks
        self.layer1 = self._make_layer(block, 16, blocks=9, stride=1)
        self.layer2 = self._make_layer(block, 32, blocks=9, stride=2)
        self.layer3 = self._make_layer(block, 64, blocks=9, stride=2)

        # Global average pool and final classification
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride):
        """Create a 'layer' consisting of `blocks` residual blocks."""
        strides = [stride] + [1]*(blocks - 1)
        layers = []
        for s in strides:
            layers.append(block(self.in_channels, out_channels, s))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

In [5]:
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc

def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc


In [6]:
model = ResNet56().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

In [7]:
#transformations for metamorphic testing
transformations = [
    transforms.RandomRotation(degrees=10),  # Small rotation
    transforms.ColorJitter(brightness=0.2),  # Slight brightness change
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))  # Small translation
]

In [8]:
import torch
import torch.nn.functional as F

def plurality_vote_prediction(model, images, transformations, device):
    """
    Given a batch of images and a list of transformations,
    compute:
    The original prediction f(x)
    The predictions for each transformed image f(T_i(x))
    Then, determine the mode (most frequent prediction) among the transformed predictions.
    If f(x) equals the mode, the prediction is accepted; otherwise, it is rejected.
    
    Returns:
      accepted_preds: a tensor of shape [batch_size] where accepted samples have their original prediction,
                      and rejected samples are marked with -1.
      accepted_mask: a boolean tensor of shape [batch_size] indicating which samples were accepted.
    """
    model.eval()
    
    # 1) Original predictions
    with torch.no_grad():
        original_out = model(images)
        _, original_preds = torch.max(F.softmax(original_out, dim=1), dim=1)  # shape [batch_size]
    
    # 2) Get predictions from each transformation (set S)
    transformed_preds_list = []
    for tf in transformations:
        # Apply the transformation on CPU (if required) and move back to device.
        x_tf = tf(images.cpu()).to(device)
        with torch.no_grad():
            out_tf = model(x_tf)
            _, preds_tf = torch.max(F.softmax(out_tf, dim=1), dim=1)
        transformed_preds_list.append(preds_tf)
    
    # 3) Compute the mode (most frequent prediction) over the transformed predictions.
    #    Note: If there are no transformations, we simply use the original predictions.
    if len(transformed_preds_list) > 0:
        stacked_preds = torch.stack(transformed_preds_list, dim=0)  # shape: [num_transforms, batch_size]
        mode_preds = stacked_preds.mode(dim=0).values  # shape: [batch_size]
    else:
        mode_preds = original_preds
    
    # 4) Determine acceptance: accepted if original_preds equals mode_preds.
    accepted_mask = (original_preds == mode_preds)
    
    # 5) Create an output: for accepted samples, return original prediction; for rejected, set to -1.
    accepted_preds = original_preds.clone()
    accepted_preds[~accepted_mask] = -1
    
    return accepted_preds, accepted_mask

In [9]:
from tqdm import tqdm

def test_with_plurality_vote(model, loader, transformations, device):
    """
    Evaluates the model on 'loader' using PluVot:
      - For each batch, it computes the plurality-voted predictions.
      - A sample is accepted if the original prediction f(x) equals the mode of the transformed predictions.
      - Rejected samples are marked with -1.
    
    It prints per-batch acceptance counts and returns:
      - The accuracy among accepted samples.
      - The overall acceptance rate.
    """
    model.eval()
    total_samples = 0
    total_accepted = 0
    correct_accepted = 0
    
    for i, (images, labels) in enumerate(tqdm(loader)):
        images, labels = images.to(device), labels.to(device)
        
        # Get PluVot predictions and acceptance mask for the batch.
        accepted_preds, accepted_mask = plurality_vote_prediction(model, images, transformations, device)
        
        batch_size = labels.size(0)
        total_samples += batch_size
        batch_accepted = accepted_mask.sum().item()
        total_accepted += batch_accepted
        
        # Calculate accuracy among accepted samples
        if batch_accepted > 0:
            batch_correct = (accepted_preds[accepted_mask] == labels[accepted_mask]).sum().item()
            correct_accepted += batch_correct
        
        print(f"Batch {i} | Accepted: {batch_accepted}/{batch_size}")
    
    overall_acceptance_rate = 100.0 * total_accepted / total_samples
    accepted_accuracy = 100.0 * correct_accepted / total_accepted if total_accepted > 0 else 0.0
    
    print(f"\nOverall Acceptance Rate: {overall_acceptance_rate:.2f}%")
    print(f"Accuracy among Accepted Samples: {accepted_accuracy:.2f}%")
    
    return accepted_accuracy, overall_acceptance_rate


In [10]:
accepted_acc, acceptance_rate = test_with_plurality_vote(model, testloader, transformations, device)
print(f"Plurality Vote Accuracy among accepted samples: {accepted_acc:.2f}%")
print(f"Overall Acceptance Rate: {acceptance_rate:.2f}%")

  3%|██▌                                                                                   | 3/100 [00:01<00:43,  2.25it/s]

Batch 0 | Accepted: 90/100
Batch 1 | Accepted: 90/100
Batch 2 | Accepted: 94/100
Batch 3 | Accepted: 85/100


  8%|██████▉                                                                               | 8/100 [00:01<00:12,  7.08it/s]

Batch 4 | Accepted: 88/100
Batch 5 | Accepted: 89/100
Batch 6 | Accepted: 94/100
Batch 7 | Accepted: 90/100
Batch 8 | Accepted: 94/100


 14%|███████████▉                                                                         | 14/100 [00:02<00:06, 12.32it/s]

...

Batch 89 | Accepted: 92/100
Batch 90 | Accepted: 89/100
Batch 91 | Accepted: 97/100
Batch 92 | Accepted: 95/100
Batch 93 | Accepted: 91/100


 98%|███████████████████████████████████████████████████████████████████████████████████▎ | 98/100 [00:06<00:00, 21.86it/s]

Batch 94 | Accepted: 94/100
Batch 95 | Accepted: 86/100
Batch 96 | Accepted: 95/100
Batch 97 | Accepted: 90/100
Batch 98 | Accepted: 96/100


100%|████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 16.05it/s]

Batch 99 | Accepted: 95/100

Overall Acceptance Rate: 90.57%
Accuracy among Accepted Samples: 8.52%
Plurality Vote Accuracy among accepted samples: 8.52%
Overall Acceptance Rate: 90.57%
